In [10]:
from collections import defaultdict
import numpy as np
import scipy as sp
import cPickle as pickle
import time
import os
from __future__ import print_function

In [11]:
start_time = time.time()
all_data = pickle.load(open("all_data.pickle", "rb"))
print(time.time() - start_time)

17.5558400154


In [19]:
all_data[9]['category']

[['Books']]

In [3]:
all_size = len(all_data)
train_data = all_data[:100000]
valid_data = all_data[900000:]

In [4]:
datum = train_data[0]
print(datum)

{'itemID': 'I572782694', 'rating': 5.0, 'helpful': {'nHelpful': 0, 'outOf': 0}, 'reviewText': 'favorite of the series...May not have been as steamy as some of the others...but the characters, their depth, and believability were amazing.  wanted to curl up with Devlin and make it all better(wink wink). an amazing series...found Laura Kate when I stumbled onto Hearts in Darkness(one of my all time faves)...this series ranks up there with my Kresley Cole and Gena Showalter favorites.', 'reviewerID': 'U243261361', 'summary': 'Loved it', 'unixReviewTime': 1399075200, 'category': [['Books']], 'reviewTime': '05 3, 2014'}


In [5]:
def get_mean_abs_error(helpfuls, helpfuls_predict):
    return np.sum(np.abs(helpfuls_predict - helpfuls.astype(float))) / helpfuls.shape[0]

In [6]:
# 1.1 alpha as global mean of helpful ratio
train_helpfuls = np.array([d['helpful']['nHelpful'] for d in train_data])
train_outofs =  np.array([d['helpful']['outOf'] for d in train_data])
alpha = np.sum(train_helpfuls) / np.sum(train_outofs.astype(float))
print('avg helpfulness ratio', alpha)

avg helpfulness ratio 0.771613890384


In [13]:
# 1.2 mean absolute error in validation set
valid_helpfuls = np.array([d['helpful']['nHelpful'] for d in valid_data])
valid_outofs =  np.array([d['helpful']['outOf'] for d in valid_data])
valid_helpfuls_predict = valid_outofs * alpha
print('mean_abs_error', get_mean_abs_error(valid_helpfuls, valid_helpfuls_predict))

mean_abs_error 0.707253269408


In [8]:
# 1.3 fit model
# theta[0] + theta[1] * #words_in_review + theta[2] * review_star_rating

###### WARING !!! WRONG SINCE SOME ENTRY IS REMOVED !!! #######
def get_feature(d):
    num_review_words = len(d['reviewText'].split())
    star = d['rating']
    return [1.0, num_review_words, star]

def get_ratio(d):
    return float(d['helpful']['nHelpful']) / float(d['helpful']['outOf'])

# linear regression, ignore devided by zero data
########## WARNING !!!! Modify here to fit either train data or all data
train_xs = np.array([get_feature(d) for d in train_data if d['helpful']['outOf'] != 0])
train_ys = np.array([get_ratio(d) for d in train_data if d['helpful']['outOf'] != 0])

theta, residuals, rank, s = np.linalg.lstsq(train_xs, train_ys)

In [19]:
theta

array([  4.58745758e-01,   1.42189987e-04,   5.97109787e-02])

In [20]:
# on train set
train_features = np.array([get_feature(d) for d in train_data])
train_helpfuls = np.array([d['helpful']['nHelpful'] for d in train_data])
train_outofs = np.array([d['helpful']['outOf'] for d in train_data])
train_helpfuls_predict = np.dot(train_features, theta) * train_outofs
print('mae', get_mean_abs_error(train_helpfuls, train_helpfuls_predict))

mae 0.683704925818


In [21]:
# on valid set
valid_features = np.array([get_feature(d) for d in valid_data])
valid_helpfuls = np.array([d['helpful']['nHelpful'] for d in valid_data])
valid_outofs = np.array([d['helpful']['outOf'] for d in valid_data])
valid_helpfuls_predict = np.dot(valid_features, theta) * valid_outofs
print('mae', get_mean_abs_error(valid_helpfuls, valid_helpfuls_predict))

mae 0.688017608167


In [ ]:
# load helpful_data.json
test_data = pickle.load(open("helpful_data.pickle", "rb"))

# on test set
test_features = np.array([get_feature(d) for d in test_data])
test_outofs = np.array([d['helpful']['outOf'] for d in test_data])
test_helpfuls_predict = np.dot(test_features, theta) * test_outofs

In [ ]:
# load 'pairs_Helpful.txt'
# get header_str and user_item_outofs
with open('pairs_Helpful.txt') as f:
    # read and strip lines
    lines = [l.strip() for l in f.readlines()]
    # stirip out the headers
    header_str = lines.pop(0)
    # get a list of user_item_ids
    user_item_outofs = [l.split('-') for l in lines]
    user_item_outofs = [[d[0], d[1], float(d[2])] for d in user_item_outofs]
    
# make sure `data.json` and `pairs_Helpful.txt` the same order
for (user_id, item_id, outof), d in zip(user_item_outofs, test_data):
    assert d['reviewerID'] == user_id
    assert d['itemID'] == item_id
    assert d['helpful']['outOf'] == outof

In [ ]:
# write to output file
f = open('predictions_Helpful.txt', 'w')
print(header_str, file=f)
for (user_id, item_id, outof), test_helpful_predict in zip(user_item_outofs, test_helpfuls_predict):
    print('%s-%s-%s,%s' % (user_id, item_id, int(outof), test_helpful_predict), file=f)
f.close()